In [5]:
%matplotlib inline
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import RobustScaler
# from ecg import ecg_feature_computation
import matplotlib.pyplot as plt
from hrvanalysis import get_time_domain_features,get_geometrical_features,get_csi_cvi_features,get_poincare_plot_features
from hrvanalysis import get_frequency_domain_features

from typing import List

import numpy as np
import scipy.signal as signal
import datetime

import numpy as np
from scipy.stats import iqr
from enum import Enum


class Quality(Enum):
    ACCEPTABLE = 1
    UNACCEPTABLE = 0

def outlier_computation(valid_rr_interval_time: list,
                        valid_rr_interval_sample: list,
                        criterion_beat_difference: float):
    """
    This function implements the rr interval outlier calculation through comparison with the criterion
    beat difference and consecutive differences with the previous and next sample

    :param valid_rr_interval_time: A python array of rr interval time
    :param valid_rr_interval_sample: A python array of rr interval samples
    :param criterion_beat_difference: A threshold calculated from the RR interval data passed

    yields: The quality of each data point in the RR interval array
    """
    standard_rr_interval_sample = valid_rr_interval_sample[0]
    previous_rr_interval_quality = Quality.ACCEPTABLE

    for i in range(1, len(valid_rr_interval_sample) - 1):

        rr_interval_diff_with_last_good = abs(standard_rr_interval_sample - valid_rr_interval_sample[i])
        rr_interval_diff_with_prev_sample = abs(valid_rr_interval_sample[i - 1] - valid_rr_interval_sample[i])
        rr_interval_diff_with_next_sample = abs(valid_rr_interval_sample[i] - valid_rr_interval_sample[i + 1])

        if previous_rr_interval_quality == Quality.UNACCEPTABLE and rr_interval_diff_with_last_good < criterion_beat_difference:
            yield (valid_rr_interval_time[i], Quality.ACCEPTABLE)
            previous_rr_interval_quality = Quality.ACCEPTABLE
            standard_rr_interval_sample = valid_rr_interval_sample[i]

        elif previous_rr_interval_quality == Quality.UNACCEPTABLE and rr_interval_diff_with_last_good > criterion_beat_difference >= rr_interval_diff_with_prev_sample and rr_interval_diff_with_next_sample <= criterion_beat_difference:
            yield (valid_rr_interval_time[i], Quality.ACCEPTABLE)
            previous_rr_interval_quality = Quality.ACCEPTABLE
            standard_rr_interval_sample = valid_rr_interval_sample[i]

        elif previous_rr_interval_quality == Quality.UNACCEPTABLE and rr_interval_diff_with_last_good > criterion_beat_difference and (
                        rr_interval_diff_with_prev_sample > criterion_beat_difference or rr_interval_diff_with_next_sample > criterion_beat_difference):
            yield (valid_rr_interval_time[i], Quality.UNACCEPTABLE)
            previous_rr_interval_quality = Quality.UNACCEPTABLE

        elif previous_rr_interval_quality == Quality.ACCEPTABLE and rr_interval_diff_with_prev_sample <= criterion_beat_difference:
            yield (valid_rr_interval_time[i], Quality.ACCEPTABLE)
            previous_rr_interval_quality = Quality.ACCEPTABLE
            standard_rr_interval_sample = valid_rr_interval_sample[i]

        elif previous_rr_interval_quality == Quality.ACCEPTABLE and rr_interval_diff_with_prev_sample > criterion_beat_difference:
            yield (valid_rr_interval_time[i], Quality.UNACCEPTABLE)
            previous_rr_interval_quality = Quality.UNACCEPTABLE

        else:
            yield (valid_rr_interval_time[i], Quality.UNACCEPTABLE)


def compute_outlier_ecg(ecg_ts,ecg_rr):
    """
    Reference - Berntson, Gary G., et al. "An approach to artifact identification: Application to heart period data."
    Psychophysiology 27.5 (1990): 586-598.

    :param ecg_rr: RR interval datastream

    :return: An annotated datastream specifying when the ECG RR interval datastream is acceptable
    """


    valid_rr_interval_sample = [i for i in ecg_rr if i > .3 and i < 2]
    valid_rr_interval_time = [ecg_ts[i] for i in range(len(ecg_ts)) if ecg_rr[i] > .3 and ecg_rr[i] < 2]
    valid_rr_interval_difference = abs(np.diff(valid_rr_interval_sample))

    # Maximum Expected Difference(MED)= 3.32* Quartile Deviation
    maximum_expected_difference = 4.5 * 0.5 * iqr(valid_rr_interval_difference)

    # Shortest Expected Beat(SEB) = Median Beat – 2.9 * Quartile Deviation
    # Minimal Artifact Difference(MAD) = SEB/ 3
    maximum_artifact_difference = (np.median(valid_rr_interval_sample) - 2.9 * .5 * iqr(
        valid_rr_interval_difference)) / 3

    # Midway between MED and MAD is considered
    criterion_beat_difference = (maximum_expected_difference + maximum_artifact_difference) / 2
    if criterion_beat_difference < .2:
        criterion_beat_difference = .2

    ecg_rr_quality_array = [(valid_rr_interval_time[0], Quality.ACCEPTABLE)]

    for data in outlier_computation(valid_rr_interval_time, valid_rr_interval_sample, criterion_beat_difference):
        ecg_rr_quality_array.append(data)
    ecg_rr_quality_array.append((valid_rr_interval_time[-1], Quality.ACCEPTABLE))
    return ecg_rr_quality_array

from typing import Tuple
from typing import List
import pandas as pd
import numpy as np

# Static name for methods params
MALIK_RULE = "malik"
KARLSSON_RULE = "karlsson"
KAMATH_RULE = "kamath"
ACAR_RULE = "acar"
CUSTOM_RULE = "custom"

def remove_ectopic_beats(rr_intervals: List[float], method: str = "malik",
                         custom_removing_rule: float = 0.2, verbose: bool = False) -> list:
    """
    RR-intervals differing by more than the removing_rule from the one proceeding it are removed.

    Parameters
    ---------
    rr_intervals : list
        list of RR-intervals
    method : str
        method to use to clean outlier. malik, kamath, karlsson, acar or custom.
    custom_removing_rule : int
        Percentage criteria of difference with previous RR-interval at which we consider
        that it is abnormal. If method is set to Karlsson, it is the percentage of difference
        between the absolute mean of previous and next RR-interval at which  to consider the beat
        as abnormal.
    verbose : bool
        Print information about ectopic beats.

    Returns
    ---------
    nn_intervals : list
        list of NN Interval
    outlier_count : int
        Count of outlier detected in RR-interval list

    References
    ----------
    .. [5] Kamath M.V., Fallen E.L.: Correction of the Heart Rate Variability Signal for Ectopics \
    and Miss- ing Beats, In: Malik M., Camm A.J.

    .. [6] Geometric Methods for Heart Rate Variability Assessment - Malik M et al
    """
    if method not in [MALIK_RULE, KAMATH_RULE, KARLSSON_RULE, ACAR_RULE, CUSTOM_RULE]:
        raise ValueError("Not a valid method. Please choose between malik, kamath, karlsson, acar.\
         You can also choose your own removing critera with custom_rule parameter.")

    if method == KARLSSON_RULE:
        nn_intervals, outlier_count = _remove_outlier_karlsson(rr_intervals=rr_intervals,
                                                               removing_rule=custom_removing_rule)

    elif method == ACAR_RULE:
        nn_intervals, outlier_count = _remove_outlier_acar(rr_intervals=rr_intervals)

    else:
        # set first element in list
        outlier_count = 0
        previous_outlier = False
        nn_intervals = [rr_intervals[0]]
        for i, rr_interval in enumerate(rr_intervals[:-1]):

            if previous_outlier:
                nn_intervals.append(rr_intervals[i + 1])
                previous_outlier = False
                continue

            if is_outlier(rr_interval, rr_intervals[i + 1], method=method,
                          custom_rule=custom_removing_rule):
                nn_intervals.append(rr_intervals[i + 1])
            else:
                nn_intervals.append(np.nan)
                outlier_count += 1
                previous_outlier = True

    if verbose :
        print("{} ectopic beat(s) have been deleted with {} rule.".format(outlier_count, method))

    return nn_intervals


def is_outlier(rr_interval: int, next_rr_interval: float, method: str = "malik",
               custom_rule: float = 0.2) -> bool:
    """
    Test if the rr_interval is an outlier

    Parameters
    ----------
    rr_interval : int
        RrInterval
    next_rr_interval : int
        consecutive RrInterval
    method : str
        method to use to clean outlier. malik, kamath, karlsson, acar or custom
    custom_rule : int
        percentage criteria of difference with previous RR-interval at which we consider
        that it is abnormal

    Returns
    ----------
    outlier : bool
        True if RrInterval is valid, False if not
    """
    if method == MALIK_RULE:
        outlier = abs(rr_interval - next_rr_interval) <= 0.2 * rr_interval
    elif method == KAMATH_RULE:
        outlier = 0 <= (next_rr_interval - rr_interval) <= 0.325 * rr_interval or 0 <= \
                  (rr_interval - next_rr_interval) <= 0.245 * rr_interval
    else:
        outlier = abs(rr_interval - next_rr_interval) <= custom_rule * rr_interval
    return outlier


def _remove_outlier_karlsson(rr_intervals: List[float], removing_rule: float = 0.2) -> Tuple[list, int]:
    """
    RR-intervals differing by more than the 20 % of the mean of previous and next RR-interval
    are removed.

    Parameters
    ---------
    rr_intervals : list
        list of RR-intervals
    removing_rule : float
        Percentage of difference between the absolute mean of previous and next RR-interval at which \
    to consider the beat as abnormal.

    Returns
    ---------
    nn_intervals : list
        list of NN Interval

    References
    ----------
    .. [7]  Automatic filtering of outliers in RR-intervals before analysis of heart rate \
    variability in Holter recordings: a comparison with carefully edited data - Marcus Karlsson, \
    Rolf Hörnsten, Annika Rydberg and Urban Wiklund
    """
    # set first element in list
    nn_intervals = [rr_intervals[0]]
    outlier_count = 0

    for i in range(len(rr_intervals)):
        # Condition to go out of loop at limits of list
        if i == len(rr_intervals)-2:
            nn_intervals.append(rr_intervals[i + 1])
            break
        mean_prev_next_rri = (rr_intervals[i] + rr_intervals[i+2]) / 2
        if abs(mean_prev_next_rri - rr_intervals[i+1]) < removing_rule * mean_prev_next_rri:
            nn_intervals.append(rr_intervals[i+1])
        else:
            nn_intervals.append(np.nan)
            outlier_count += 1
    return nn_intervals, outlier_count


def _remove_outlier_acar(rr_intervals: List[float], custom_rule=0.2) -> Tuple[list, int]:
    """
    RR-intervals differing by more than the 20 % of the mean of last 9 RrIntervals
    are removed.

    Parameters
    ---------
    rr_intervals : list
        list of RR-intervals
    custom_rule : int
        percentage criteria of difference with mean of  9 previous RR-intervals at
        which we consider that RR-interval is abnormal. By default, set to 20 %

    Returns
    ---------
    nn_intervals : list
        list of NN Interval

    References
    ----------
    .. [8] Automatic ectopic beat elimination in short-term heart rate variability measurements \
    Acar B., Irina S., Hemingway H., Malik M.
    """
    nn_intervals = []
    outlier_count = 0
    for i, rr_interval in enumerate(rr_intervals):
        if i < 9:
            nn_intervals.append(rr_interval)
            continue
        acar_rule_elt = np.nanmean(nn_intervals[-9:])
        if abs(acar_rule_elt - rr_interval) < custom_rule * acar_rule_elt:
            nn_intervals.append(rr_interval)
        else:
            nn_intervals.append(np.nan)
            outlier_count += 1
    return nn_intervals, outlier_count



def lomb(time_stamps:List,
         samples:List,
         low_frequency: float,
         high_frequency: float):
    """
   : Lomb–Scargle periodogram implementation
    :param data: List[DataPoint]
    :param high_frequency: float
    :param low_frequency: float
    :return lomb-scargle pgram and frequency values
    """

    frequency_range = np.linspace(low_frequency, high_frequency, len(time_stamps))
    result = signal.lombscargle(time_stamps, samples, frequency_range)
    return result, frequency_range


def heart_rate_power(power: np.ndarray,
                     frequency: np.ndarray,
                     low_rate: float,
                     high_rate: float):
    """
    Compute Heart Rate Power for specific frequency range
    :param power: np.ndarray
    :param frequency: np.ndarray
    :param high_rate: float
    :param low_rate: float
    :return: sum of power for the frequency range
    """
    result_power = float(0.0)
    for i, value in enumerate(power):
        if low_rate <= frequency[i] <= high_rate:
            result_power += value
    return result_power






def ecg_feature_computation(timestamp:list,
                            value:list,
                            low_frequency: float = 0.01,
                            high_frequency: float = 0.7,
                            low_rate_vlf: float = 0.0009,
                            high_rate_vlf: float = 0.04,
                            low_rate_hf: float = 0.15,
                            high_rate_hf: float = 0.4,
                            low_rate_lf: float = 0.04,
                            high_rate_lf: float = 0.15):
    """
    ECG Feature Implementation. The frequency ranges for High, Low and Very low heart rate variability values are
    derived from the following paper:
    'Heart rate variability: standards of measurement, physiological interpretation and clinical use'
    :param high_rate_lf: float
    :param low_rate_lf: float
    :param high_rate_hf: float
    :param low_rate_hf: float
    :param high_rate_vlf: float
    :param low_rate_vlf: float
    :param high_frequency: float
    :param low_frequency: float
    :param datastream: DataStream
    :param window_size: float
    :param window_offset: float
    :return: ECG Feature DataStreams
    """


    # perform windowing of datastream


    # initialize each ecg feature array

    rr_variance_data = []
    rr_mean_data = []
    rr_median_data = []
    rr_80percentile_data = []
    rr_20percentile_data = []
    rr_quartile_deviation_data = []
    rr_HF_data = []
    rr_LF_data = []
    rr_VLF_data = []
    rr_LF_HF_data = []
    rr_heart_rate_data = []

    # iterate over each window and calculate features


    reference_data = value

    rr_variance_data.append(np.var(reference_data))

    power, frequency = lomb(time_stamps=timestamp,samples=value,low_frequency=low_frequency, high_frequency=high_frequency)

    rr_VLF_data.append(heart_rate_power(power, frequency, low_rate_vlf, high_rate_vlf))

    rr_HF_data.append(heart_rate_power(power, frequency, low_rate_hf, high_rate_hf))

    rr_LF_data.append(heart_rate_power(power,frequency,low_rate_lf,high_rate_lf))

    if heart_rate_power(power, frequency, low_rate_hf, high_rate_hf) != 0:
        lf_hf = float(heart_rate_power(power, frequency, low_rate_lf, high_rate_lf) / heart_rate_power(power,
                                                                                                       frequency,
                                                                                                       low_rate_hf,
                                                                                                       high_rate_hf))
        rr_LF_HF_data.append(lf_hf)
    else:
        rr_LF_HF_data.append(0)

    rr_mean_data.append(np.mean(reference_data))
    rr_median_data.append(np.median(reference_data))
    rr_quartile_deviation_data.append((0.5*(np.percentile(reference_data, 75) - np.percentile(reference_data,25))))
    rr_heart_rate_data.append(60/np.median(reference_data))

    return [rr_variance_data[0],rr_quartile_deviation_data[0],\
           rr_VLF_data[0], rr_LF_data[0],rr_HF_data[0], rr_LF_HF_data[0],\
           rr_mean_data[0], rr_median_data[0],\
           np.percentile(value,80),np.percentile(value,20),\
           rr_heart_rate_data[0]]

def get_label(user_data,st,et):
    label = 2
    for k in range(user_data.shape[0]):
        if st>=user_data[k,0] and et<=user_data[k,1]:
            label = user_data[k,2]

    return label
def get_ecg_windowss(rr_interval):
    window_col,ts_col = [],[]
    ts_array = np.arange(rr_interval[0,0],rr_interval[-1,0],30000)
    for t in ts_array:
        index = np.where((rr_interval[:,0]>=t)&(rr_interval[:,0]<=t+60000))[0]
        if len(index)<40:
            continue
        rr_temp = rr_interval[index,:]
#         if len(np.where((rr_temp[:,1]<-3)|(rr_temp[:,1]>3))[0])/len(index) > .1:
#             continue
        window_col.append(rr_temp)
        ts_col.append(t)
    return window_col,ts_col

def get_rr_features(a):
    return np.array([np.var(a),iqr(a),np.mean(a),np.median(a),np.percentile(a,80),np.percentile(a,20),60000/np.median(a)])


def combine_data_sobc(window_col,ts_col,label_data,participant):
    feature_matrix = []
    user_col = []
    label_col = []
    for i,item in enumerate(window_col):
#         if get_label(label_data,ts_col[i],ts_col[i]+50000) not in [1,0]:
#             continue
#         feature = ecg_feature_computation(item[:,0]/1000,item[:,1]/1000)
        feature = get_rr_features(item[:,1])
#         f = get_time_domain_features(item[:,1])
#         feature.extend(list(f.values()))
#         f = get_csi_cvi_features(item[:,1])
#         feature.extend(list(f.values()))
#         f = get_poincare_plot_features(item[:,1])
#         feature.extend(list(f.values()))
        feature_matrix.append(np.array(feature).reshape(-1,7))
        user_col.append(participant)
        label_col.append(get_label(label_data,ts_col[i],ts_col[i]+50000))
    return np.array(feature_matrix).reshape(-1,11),user_col,label_col
def get_2_sec_ts(rr_ppg_int):
    m = np.mean(rr_ppg_int[:,1])
    s = np.std(rr_ppg_int[:,1])
#     rr_ppg_int = rr_ppg_int[np.where((rr_ppg_int[:,1]>=m-3*s)&(rr_ppg_int[:,1]<=m+3*s))[0],:]
    ts_array = np.arange(rr_ppg_int[0,0],rr_ppg_int[-1,0],1000)
    rr_interval = np.zeros((0,2))
    for t in ts_array:
        index = np.where((rr_ppg_int[:,0]>=t-2000)&(rr_ppg_int[:,0]<=t+2000))[0]
        if len(index) < 1:
            continue
        rr_interval = np.concatenate((rr_interval,np.array([t,np.mean(rr_ppg_int[index,1])]).reshape(-1,2)))
    return rr_interval
from scipy.stats.mstats_basic import winsorize

path = '/home/jupyter/mullah/Test/data_yield/data/sobc_2nd_chance//'
participants = os.listdir(path)
from ecgdetectors import Detectors
detectors = Detectors(100)
X,y,groups = np.zeros((0,7)),[],[]
for participant in participants:
    file_list = os.listdir(path+'/'+participant)
    for file in file_list:
        final_path = path+'/'+participant+'/'+file+'/'
        try:
            if 'ecg_rr_pan_tomkins1.csv' not in os.listdir(final_path):
                ecg_final_data = pd.read_csv(final_path+'ecg_final_data.csv',header=None,sep=',').values
                rpeaks = detectors.pan_tompkins_detector(ecg_final_data[:,2])
                rpeaks = np.array(rpeaks)
                rpeak_ts = ecg_final_data[rpeaks,0]
                ecg_rr_ts = rpeak_ts[1:]
                ecg_rr_val = np.diff(rpeak_ts)
                index = np.where((ecg_rr_val>=300)&(ecg_rr_val<=2000))[0]
                ecg_rr_ts = ecg_rr_ts[index]
                ecg_rr_val = ecg_rr_val[index]
#                 rr = remove_ectopic_beats(list(ecg_rr_val),'acar')
#                 ecg_rr_val = ecg_rr_val[~np.isnan(rr)]
#                 ecg_rr_ts = ecg_rr_ts[~np.isnan(rr)]
                outlier = compute_outlier_ecg(ecg_rr_ts/1000,ecg_rr_val/1000)
                index = []
                for ind,tup in enumerate(outlier):
                    if tup[1]==Quality.ACCEPTABLE:
                        index.append(ind)
                index = np.array(index)
                ecg_rr_ts = ecg_rr_ts[index]
                ecg_rr_val = ecg_rr_val[index]
                ecg_rr = np.zeros((len(ecg_rr_ts),2))
                ecg_rr[:,0] = ecg_rr_ts
                ecg_rr[:,1] = ecg_rr_val
                df = pd.DataFrame(ecg_rr, columns= ['time', 'rr'])
                df.to_csv(final_path+'ecg_rr_pan_tomkins1.csv',index=False,header=False)
            else:
                ecg_rr = pd.read_csv(final_path+'ecg_rr_pan_tomkins1.csv',header=None,sep=',').values
            if ecg_rr.shape[0]<500:
                continue
            winsor_limit = .05
            ecg_rr[:,1] = winsorize(ecg_rr[:,1],limits=[winsor_limit,winsor_limit])
#             plt.plot(ecg_rr[:,1])
#             plt.show()
            label_data = pd.read_csv(final_path+'label_data.csv',header=None,sep=',').values
#             print(label_data.shape,participant)
#             mean_col = []
#             std_col = []
#             i = 0
#             while i < len(ecg_rr):
#                 start_ts = ecg_rr[i,0]
#                 j = i
#                 while j<len(ecg_rr) and ecg_rr[j,0]-start_ts <= 60000:
#                     j+=1
#                 mean_col.append(np.mean(ecg_rr[i:j+1,1]))
#                 std_col.append(np.std(ecg_rr[i:j+1,1]))
#                 i=j
#             m = np.percentile(mean_col,70)
#             s = np.percentile(std_col,30)

#             ecg_rr = get_2_sec_ts(ecg_rr)
#             index = np.where(ecg_rr[:,0]<ecg_rr[0,0]+10*60*1000)[0]
           
            m = np.mean(ecg_rr[:,1])
            s = np.std(ecg_rr[:,1])
            if s==0:
                continue
#             ecg_rr[:,1] = (ecg_rr[:,1]-m)/s 
            
#             ecg_rr[np.where((ecg_rr[:,1]>=3))[0],1] = 3
#             ecg_rr[np.where((ecg_rr[:,1]<=-3))[0],1] = -3
#             ecg_rr[:,1] = RobustScaler(quantile_range=(20,80)).fit_transform(ecg_rr[:,1:]).reshape(len(ecg_rr),)
            window_col,ts_col = get_ecg_windowss(ecg_rr)
            feature_matrix,user_col,label_col = combine_data_sobc(window_col,ts_col,label_data,participant)
            temp = np.array(label_col)
#             if len(temp[temp==1])/len(temp[temp==0]) < .5:
#                 continue
#             if len(temp[temp==1])<5 or len(temp[temp==1])<10:
#                 continue
            labels = np.array(label_col)
#             if len(labels[labels==1])<5:
#                 continue
            X = np.concatenate((X,feature_matrix))
            y.extend(label_col)
            groups.extend(user_col)
            print(X.shape,len(np.unique(groups)))
        except Exception as e:
            print(e)
            e

# winsor_limit = .02
# for k in range(X.shape[1]):
#     X[:,k] = winsorize(X[:,k],limits=[winsor_limit,winsor_limit])

cannot reshape array of size 1547 into shape (11)
File b'/home/jupyter/mullah/Test/data_yield/data/sobc_2nd_chance///eb672766-4202-4f8d-8cf3-8bf15425d5b0/20180406/ecg_final_data.csv' does not exist
File b'/home/jupyter/mullah/Test/data_yield/data/sobc_2nd_chance///b191d2d6-a307-4a50-9661-0d9db3d8562c/20180228/ecg_final_data.csv' does not exist
cannot reshape array of size 966 into shape (11)
cannot reshape array of size 1575 into shape (11)
cannot reshape array of size 840 into shape (11)
cannot reshape array of size 1575 into shape (11)
cannot reshape array of size 1575 into shape (11)
cannot reshape array of size 1435 into shape (11)
cannot reshape array of size 1498 into shape (11)
cannot reshape array of size 1792 into shape (11)
File b'/home/jupyter/mullah/Test/data_yield/data/sobc_2nd_chance///50af9985-2197-4e1e-bee3-556816f446a1/20180501/ecg_final_data.csv' does not exist
cannot reshape array of size 1085 into shape (11)
cannot reshape array of size 1484 into shape (11)
cannot r

KeyboardInterrupt: 

In [11]:
print(len(np.unique(groups)))

31


In [12]:
from sklearn.preprocessing import StandardScaler
y = np.array(y)
groups = np.array(groups)
y = y[~np.isnan(X).any(axis=1)]
groups = groups[~np.isnan(X).any(axis=1)]
X = X[~np.isnan(X).any(axis=1)]
print(X.shape,len(np.unique(groups)))
for user in np.unique(groups):
    index = np.where(groups==user)[0]
    X[index,:] = StandardScaler().fit_transform(X[index,:])
X[X>4] = 4
X[X<-4] = -4
index = np.where(y<2)[0]
X,y,groups = X[index],y[index],groups[index]
print(X.shape,len(np.unique(groups)))

(5313, 11) 31
(3162, 11) 31


In [19]:
from sklearn.decomposition import PCA
from pprint import pprint
from sklearn.metrics import f1_score
from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.svm import SVC
m = len(np.where(y==0)[0])
n = len(np.where(y>0)[0])
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix,f1_score,precision_score,recall_score,accuracy_score
import itertools
from sklearn.model_selection import ParameterGrid, cross_val_predict, GroupKFold,GridSearchCV
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
from joblib import Parallel,delayed

delta = 0.1

paramGrid = {'rf__kernel': ['rbf'],
             'rf__C': [11],
             'rf__gamma': [np.power(2,np.float(x)) for x in np.arange(-6, -2, .5)],
             'rf__class_weight': [{0: w, 1: 1 - w} for w in [.2,.3]],
             'rf__probability':[True]
}
pca = PCA(n_components=4)
clf = Pipeline([('rf', SVC())])
gkf = GroupKFold(n_splits=len(np.unique(groups)))
grid_search = GridSearchCV(clf, paramGrid, n_jobs=-1,cv=list(gkf.split(X,y,groups=groups)),
                           scoring='accuracy',verbose=5)
grid_search.fit(X[:,:],y)

print("Best parameter (CV score=%0.3f):" % grid_search.best_score_)
print(grid_search.best_params_)

Fitting 31 folds for each of 16 candidates, totalling 496 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 48 concurrent workers.
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   10.3s
[Parallel(n_jobs=-1)]: Done 354 tasks      | elapsed:   19.1s
[Parallel(n_jobs=-1)]: Done 496 out of 496 | elapsed:   25.3s finished


Best parameter (CV score=0.852):
{'rf__C': 11, 'rf__class_weight': {0: 0.3, 1: 0.7}, 'rf__gamma': 0.03125, 'rf__kernel': 'rbf', 'rf__probability': True}


In [20]:
import warnings
from sklearn.metrics import classification_report
warnings.filterwarnings('ignore')
clf = grid_search.best_estimator_
y_pred = cross_val_predict(clf,X,y,cv=gkf.split(X,y,groups=groups))
print(confusion_matrix(y,y_pred),classification_report(y,y_pred))

[[2044  286]
 [ 181  651]]               precision    recall  f1-score   support

         0.0       0.92      0.88      0.90      2330
         1.0       0.69      0.78      0.74       832

   micro avg       0.85      0.85      0.85      3162
   macro avg       0.81      0.83      0.82      3162
weighted avg       0.86      0.85      0.85      3162



In [21]:
import pickle
print(clf)
clf.fit(X,y)
pickle.dump(clf,open('/home/jupyter/mullah/cc3/ecg_model_feature_standardization_sobc1.p','wb'))

Pipeline(memory=None,
     steps=[('rf', SVC(C=11, cache_size=200, class_weight={0: 0.3, 1: 0.7}, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.03125, kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])


In [ ]:
# from sklearn.metrics import plot_confusion_matrix
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

mats = np.array([[794 ,220],[170 ,497]])
sns.set(font_scale=1.5)
df_cm = pd.DataFrame(mats, index = [i for i in ['Not Stress','Stress']],
                  columns = [i for i in ['Not Stress','Stress']])
plt.figure(figsize = (10,7))
sns.heatmap(df_cm, annot=True,fmt='g',annot_kws={"fontsize":28})
plt.show()

In [ ]:
clf.fit(X,y)
temp = np.array([1.8071820221852526,
                 0.12102575000797881,
                 1.1841492817462929,
                 0.9287861011371492,
                 1.9107735231269358,
                 -0.9425789234303202,
                 1.0006703240454398,
                 1.3585399583852928,
                 1.3651308639025757,
                 1.0414474739221673,
                 -1.339951252451994]).reshape(-1,11)
print(clf.decision_function(temp),clf.probA_,clf.probB_)
print(clf.predict(temp),clf.predict_proba(temp))

In [ ]:
df = pd.DataFrame(X,columns=['var', 
                             'iqr', 
                             'vlf',  
                             'lf',
                             'hf',
                             'lfhf',
                             'mean',
                             'median',
                             '80th',
                             '20th',
                             'heartrate'])
df['stress_probability'] = list(clf.predict_proba(X)[:,1])
df['label'] = list(y)
print(confusion_matrix(y,df['label']))

In [ ]:
df.to_csv('stress_all_data.csv')

In [ ]:
decisions = cross_val_predict(clf,X,y,cv=list(gkf.split(X,y,groups=groups)),method='decision_function')

In [ ]:
delta = 0.2
# paramGrid = ParameterGrid({'C': np.logspace(-3,3,20),
#               'class_weight': [{0: w, 1: 1 - w} for w in np.arange(0.0, 1.0, delta)]})
paramGrid = {
             'C': np.logspace(.1,2,10),
             'class_weight': [{0: w, 1: 1 - w} for w in np.arange(0.0, 1.0, delta)],
}
clf = LogisticRegression()
gkf = GroupKFold(n_splits=len(np.unique(groups)))
grid_search = GridSearchCV(clf, paramGrid, n_jobs=-1,cv=list(gkf.split(X,y,groups=groups)),
                           scoring='f1',verbose=5)
grid_search.fit(decisions.reshape(-1,1),y)

In [ ]:
# clf = CalibratedClassifierCV(clf,cv =gkf.split(X,y,groups=groups), method='isotonic')
# clf.fit(X,y)
# # list(zip(clf.steps[0][1].mean_,clf.steps[0][1].scale_))
# plt.figure(figsize=(16,8))

# plt.plot(y)
# plt.plot(clf.predict_proba(X)[:,1])
# plt.show()
# print(f1_score(y,clf.predict(X)))
import pickle
from sklearn import metrics
# clf.set_params(rf__probability=True)
clf.fit(X,y)
# clf.score(X)
# print(accuracy_score(y,clf.predict(X1)))
# pickle.dump(clf,open('/home/jupyter/mullah/cc3/ecg_model.p','wb'))
def f1Bias_scorer_CV(probs, y, ret_bias=False):
    precision, recall, thresholds = metrics.precision_recall_curve(y, probs)

    f1 = 0.0
    for i in range(0, len(thresholds)):
        if not (precision[i] == 0 and recall[i] == 0):
            f = 2 * (precision[i] * recall[i]) / (precision[i] + recall[i])
            if f > f1:
                f1 = f
                bias = thresholds[i]

    if ret_bias:
        return f1, bias
    else:
        return f1
score, bias = f1Bias_scorer_CV(clf.predict_proba(X)[:,1], y, True)
print(score,bias,confusion_matrix(y,clf.predict(X)))
plt.plot(y[:])
plt.plot(clf.predict_proba(X)[:,1])

In [ ]:
import pandas as pd
df = pd.DataFrame(X,columns=['var','iqr','vlf','lf','hf','lfhf','mean','median','80th','20th','heartrate'])
df['stress'] = y
# df1 = pickle.load(open('./stress_code/feature.p','rb'))
%matplotlib inline
import seaborn as sns
# sns.set(style="ticks", color_codes=True)
# g = sns.pairplot(df)
df['median'].hist(bins=20)
# plt.title('Original-Heart rate')
# plt.savefig('./stress_code/2.png',dps=1e6)

iris_X = df[df.columns.difference(["stress"])]
iris_y = df["stress"]
iris_X.to_csv('stress_feature_data.csv')
iris_y.to_csv('stress_label_data.csv')

In [ ]:
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import StandardScaler
from sklearn2pmml.decoration import ContinuousDomain

column_preprocessor = DataFrameMapper([
    (['var','iqr','vlf','lf','hf','lfhf','mean','median','80th','20th','heartrate'], [ContinuousDomain(), StandardScaler()])
])

In [ ]:
from sklearn2pmml.pipeline import PMMLPipeline
# classifier = LogisticRegression(C=0.018329807108324356, class_weight={0: 0.5, 1: 0.5},
#           dual=False, fit_intercept=True, intercept_scaling=1,
#           max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
#           random_state=None, solver='warn', tol=0.0001, verbose=0,
#           warm_start=False)
classifier = LogisticRegression(C=0.001,
          class_weight={0: 0.6000000000000001, 1: 0.3999999999999999},
          dual=False, fit_intercept=True, intercept_scaling=1,
          max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
          random_state=None, solver='warn', tol=0.0001, verbose=0,
          warm_start=False)
pipeline = PMMLPipeline([
    ("columns", column_preprocessor),
    ("classifier", classifier)
])

In [ ]:
pipeline.fit(iris_X, iris_y)

In [ ]:
print(f1_score(y,pipeline.predict(iris_X)),confusion_matrix(y,pipeline.predict(iris_X)))

In [ ]:
pipeline.verify(iris_X.sample(n = 15)),pipeline.predict_proba(iris_X)[:3]

In [ ]:
from sklearn.externals import joblib
joblib.dump(pipeline, "pipeline.pkl.z", compress = 9)

In [ ]:
clf,pipeline

In [ ]:
print(f1_score(y,clf.predict(X)))

In [ ]:
import pickle
pickle.dump([clf,X,y],open('clf_and_data.p','wb'))

In [ ]:
X1 = preprocessing.StandardScaler().fit_transform(X)

In [ ]:
# X = preprocessing.StandardScaler().fit_transform(X)
import pickle
clf.fit(X1,y)
print(accuracy_score(y,clf.predict(X1)))
pickle.dump(clf,open('ecg_model.p','wb'))
pickle.dump(X1,open('X.p','wb'))
pickle.dump(y,open('y.p','wb'))

In [ ]:
### from sklearn.decomposition import PCA
from pprint import pprint
import numpy as np
# import parfit.parfit as pf
from sklearn.base import clone, is_classifier
from sklearn.metrics import f1_score
from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.svm import SVC
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix,f1_score,precision_score,recall_score,accuracy_score,classification_report
import itertools
from sklearn.model_selection import ParameterGrid, cross_val_predict, GroupKFold,GridSearchCV
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import pandas as pd
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
import warnings
from sklearn.model_selection import check_cv
from sklearn.externals.joblib import Parallel, delayed
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, ParameterSampler, ParameterGrid
from sklearn.utils.validation import _num_samples, indexable
warnings.filterwarnings('ignore')
from sklearn import metrics

def Twobias_scorer_CV(probs, y, ret_bias=False):
    db = np.transpose(np.vstack([np.array(probs).reshape(-1), np.array(y).reshape(-1)]))
    db = db[np.argsort(db[:, 0]), :]

    pos = np.sum(y == 1)
    n = len(y)
    neg = n - pos
    tp, tn = pos, 0
    lost = 0

    optbias = []
    minloss = 1

    for i in range(n):
        #		p = db[i,1]
        if db[i, 1] == 1:  # positive
            tp -= 1.0
        else:
            tn += 1.0

        # v1 = tp/pos
        #		v2 = tn/neg
        if tp / pos >= 0.95 and tn / neg >= 0.95:
            optbias = [db[i, 0], db[i, 0]]
            continue

        running_pos = pos
        running_neg = neg
        running_tp = tp
        running_tn = tn

        for j in range(i + 1, n):
            #			p1 = db[j,1]
            if db[j, 1] == 1:  # positive
                running_tp -= 1.0
                running_pos -= 1
            else:
                running_neg -= 1

            lost = (j - i) * 1.0 / n
            if running_pos == 0 or running_neg == 0:
                break

            # v1 = running_tp/running_pos
            #			v2 = running_tn/running_neg

            if running_tp / running_pos >= 0.95 and running_tn / running_neg >= 0.95 and lost < minloss:
                minloss = lost
                optbias = [db[i, 0], db[j, 0]]

    if ret_bias:
        return -minloss, optbias
    else:
        return -minloss
def cv_fit_and_score(estimator, X, y, scorer, parameters, cv):
    """Fit estimator and compute scores for a given dataset split.
    Parameters
    ----------
    estimator : estimator object implementing 'fit'
        The object to use to fit the data.
    X : array-like of shape at least 2D
        The data to fit.
    y : array-like, optional, default: None
        The target variable to try to predict in the case of
        supervised learning.
    scorer : callable
        A scorer callable object / function with signature
        ``scorer(estimator, X, y)``.
    parameters : dict or None
        Parameters to be set on the estimator.
    cv:	Cross-validation fold indeces
    Returns
    -------
    score : float
        CV score on whole set.
    parameters : dict or None, optional
        The parameters that have been evaluated.
    """
    estimator.set_params(**parameters)
    cv_probs_ = cross_val_probs(estimator, X, y, cv)
    score = scorer(cv_probs_, y)

    return [score, parameters]  # scoring_time
    
def cross_val_probs(estimator, X, y, cv):
    probs = np.zeros(len(y))
    probs = cross_val_predict(estimator, X, y, cv=cv,method='predict_proba')[:,1]
#     for train, test in cv:
#         temp = estimator.fit(X[train], y[train]).predict_proba(X[test])
#         probs[test] = temp[:, 1]

    return probs

def f1Bias_scorer_CV(probs, y, ret_bias=False):
    precision, recall, thresholds = metrics.precision_recall_curve(y, probs)

    f1 = 0.0
    for i in range(0, len(thresholds)):
        if not (precision[i] == 0 and recall[i] == 0):
            f = 2 * (precision[i] * recall[i]) / (precision[i] + recall[i])
            if f > f1:
                f1 = f
                bias = thresholds[i]

    if ret_bias:
        return f1, bias
    else:
        return f1
    
class ModifiedGridSearchCV(GridSearchCV):
    def __init__(self, estimator, param_grid, scoring=None, fit_params=None,
                 n_jobs=1, iid=True, refit=True, cv=None, verbose=0,
                 pre_dispatch='2*n_jobs', error_score='raise'):

        super(ModifiedGridSearchCV, self).__init__(
                estimator, param_grid, scoring, fit_params, n_jobs, iid,
                refit, cv, verbose, pre_dispatch, error_score)

    def fit(self, X, y):
        """Actual fitting,  performing the search over parameters."""

        parameter_iterable = ParameterGrid(self.param_grid)

        estimator = self.estimator
        cv = self.cv

        n_samples = _num_samples(X)
        X, y = indexable(X, y)

        if y is not None:
            if len(y) != n_samples:
                raise ValueError('Target variable (y) has a different number '
                                 'of samples (%i) than data (X: %i samples)'
                                 % (len(y), n_samples))
#         cv = check_cv(cv, X, y, classifier=is_classifier(estimator))

        if self.verbose > 0:
#             if isinstance(parameter_iterable, Sized):
            n_candidates = len(parameter_iterable)
            print("Fitting {0} folds for each of {1} candidates, totalling"
                  " {2} fits".format(len(cv), n_candidates,
                                     n_candidates * len(cv)))

        base_estimator = clone(self.estimator)

        pre_dispatch = self.pre_dispatch

        out = Parallel(
                n_jobs=self.n_jobs, verbose=self.verbose,
                pre_dispatch=pre_dispatch
        )(
                delayed(cv_fit_and_score)(clone(base_estimator), X, y, self.scoring,
                                          parameters, cv=cv)
                for parameters in parameter_iterable)
#         print(out)
        best = sorted(out,key=lambda x: x[0], reverse=True)[0]
        self.best_params_ = best[1]
        self.best_score_ = best[0]

        if self.refit:
            # fit the best estimator using the entire dataset
            # clone first to work around broken estimators
            best_estimator = clone(base_estimator).set_params(
                    **best[1])
#             if y is not None:
#                 best_estimator.fit(X, y, **self.fit_params)
#             else:
#                 best_estimator.fit(X, **self.fit_params)
            self.best_estimator_ = best_estimator

        return self

In [ ]:
gkf = GroupKFold(n_splits=len(np.unique(groups)))
X1 = preprocessing.StandardScaler().fit_transform(X)
delta = 0.1
parameters1 = {'kernel': ['rbf'],
              'C': np.logspace(0,2,2),
              'gamma': np.logspace(-9,9,10),
              'class_weight': [{0: w, 1: 1 - w} for w in np.arange(0.0, 1.0, delta)],
              'probability':[True],
              'verbose':[False],
              'cache_size':[2000]}
parameters = {
    'min_samples_leaf': [4],
    'max_features': [.7,1],
    'n_estimators': [100,200,300],
    'n_jobs': [-1],
    'criterion':['gini','entropy'],
    'class_weight': [{0: w, 1: 1 - w} for w in np.arange(0.0, 1.0, delta)],
    'random_state': [42]
       }
svc = SVC()
# svc = RandomForestClassifier()
# grid_search = GridSearchCV(svc,parameters, cv=gkf.split(X1,y,groups=groups), 
#              n_jobs=-1, scoring='f1', verbose=1, iid=False)
# clf = Pipeline([('sts',StandardScaler()),('clf',svc)])
grid_search = ModifiedGridSearchCV(svc, parameters1, cv=list(gkf.split(X1,y,groups=groups)),
                                   n_jobs=20, scoring=f1Bias_scorer_CV, verbose=1, iid=False)
grid_search.fit(X1,y)
clf = grid_search.best_estimator_
clf

In [ ]:
m = len(np.where(y==0)[0])
n = len(np.where(y>0)[0])
clf.probability = True
CV_probs = cross_val_probs(clf, X1, y, gkf.split(X1,y,groups=groups))
# score, bias = Twobias_scorer_CV(CV_probs, y, True)
score, bias = f1Bias_scorer_CV(CV_probs, y, True)
predicted = np.asarray(CV_probs >= bias, dtype=np.int)
classified = range(n)
print(score,bias)

f = np.zeros((len(y),2))

data = pd.DataFrame()
print(metrics.classification_report(y, predicted))
print(metrics.confusion_matrix(y, predicted))

data['groups'] = groups
data['original'] = [[i] for i in y]
data['predicted'] = [[i] for i in predicted]
f_scores = []
data = data.groupby('groups').sum()
for i in range(data.shape[0]):
    f_scores.append(f1_score(data['original'][i],data['predicted'][i]))
print(np.median(f_scores))

In [ ]:
from sklearn.metrics import confusion_matrix,f1_score,precision_score,recall_score
import itertools
from sklearn.model_selection import ParameterGrid, cross_val_predict, GroupKFold,GridSearchCV
def plot_confusion_matrix(cm, classes=['Not Stress','Stress'],
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.figure()
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.savefig('result.png')
    plt.show()
gkf = GroupKFold(n_splits=len(np.unique(groups)))
predicted = cross_val_predict(clf, X, y, cv=gkf.split(X,y,groups=groups),n_jobs=24)
plot_confusion_matrix(confusion_matrix(y,predicted))
print(f1_score(y,predicted),precision_score(y,predicted),recall_score(y,predicted))

In [ ]:
from sklearn.feature_selection import RFECV
svc = predictor
gkf = GroupKFold(n_splits=len(np.unique(groups)))
rfecv = RFECV(estimator=svc, step=1, cv=gkf.split(X,y,groups=groups),
              scoring='f1',n_jobs=24)
rfecv.fit(X, y)

print("Optimal number of features : %d" % rfecv.n_features_)
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()

In [ ]:
np.power(2,-1)

In [ ]:
import xgboost